In [0]:
WITH GM_curr_owner AS (
SELECT gm_person_real_id
FROM (
    SELECT 
        gm_person_real_id, 
        indiv_id, 
        nameplate
    FROM infact_dev.marsci.output_develop_fs_pronghorn_xref
    RIGHT JOIN (
        SELECT 
            acq.individual_business_id, 
            acq.vehicle_status_cd, 
            acq.acquisition_dt, 
            LOWER(info.umf_xref_finc_brand) AS nameplate
        FROM acquire.vehicle_silver.acquire.vehicle_silver.customer_vin_relation AS acq
        LEFT JOIN reporting.vehicle360.fs_veh_ident AS info
            ON acq.vin = info.veh_id_nbr
        WHERE 1 = 1
          AND acq.vehicle_status_cd LIKE '%SR%'         -- Still retained filter
          AND acq.individual_business_id LIKE '1%'        -- Retail sale filter
          AND ctry_of_sale = 'US'
    ) AS sub
        ON indiv_id = individual_business_id
)
WHERE nameplate IS NOT NULL
GROUP BY gm_person_real_id
)

SELECT COUNT(*) FROM GM_curr_owner


SELECT 
    DISTINCT vehicle_status_cd
FROM acquire.vehicle_silver.customer_vin_relation AS acq
LEFT JOIN reporting.vehicle360.fs_veh_ident AS info
    ON acq.vin = info.veh_id_nbr
WHERE 1 = 1
--   AND acq.vehicle_status_cd LIKE '%SR%'         -- Bought new filter
    AND acq.individual_business_id LIKE '1%'        -- Retail sale filter
    AND ctry_of_sale = 'US'

, vehicles_owner AS (
SELECT GM_PERSON_REALID AS gm_person_real_id 
FROM dataproducts_dev.bronze_acxiom.gm_consumer_list
WHERE 1 = 1
AND PM021107 > 1
AND snapshot = (
SELECT MAX(snapshot) FROM
dataproducts_dev.bronze_acxiom.gm_consumer_list
)
)

SELECT COUNT(GM_PERSON_REALID) 
FROM dataproducts_dev.bronze_acxiom.gm_consumer_list
WHERE 1 = 1
AND snapshot = (
SELECT MAX(snapshot) FROM
dataproducts_dev.bronze_acxiom.gm_consumer_list
)

, non_GM_curr_owner_only AS (
SELECT *
FROM vehicles_owner
WHERE gm_person_real_id
NOT IN 
(
SELECT gm_person_real_id
FROM GM_curr_owner
WHERE gm_person_real_id IS NOT NULL  
)
)


, non_GM_curr_owner AS (
SELECT DISTINCT GM_PERSON_REALID AS gm_person_real_id 
FROM dataproducts_dev.bronze_acxiom.gm_consumer_list
WHERE 1 = 1
AND (
PM020689 = 1	-- Polk - Owner - Volvo
OR PM020684 = 1	-- Polk - Owner - Subaru
OR PM020650 = 1	-- Polk - Owner - Audi
OR PM020687 = 1	-- Polk - Owner - Toyota
OR PM020666 = 1	-- Polk - Owner - Jaguar
OR PM020682 = 1	-- Polk - Owner - Saab
OR PM020680 = 1	-- Polk - Owner - Porsche
OR PM020686 = 1	-- Polk - Owner - Tesla
OR PM021425 = 1	-- Polk - Owner - Rolls Royce
OR PM020685 = 1	-- Polk - Owner - Suzuki
OR PM020657 = 1	-- Polk - Owner - Fiat
OR PM020649 = 1	-- Polk - Owner - Alfa Romeo
OR PM020661 = 1	-- Polk - Owner - Honda
OR PM020672 = 1	-- Polk - Owner - Mazda
OR PM020675 = 1	-- Polk - Owner - Mitsubishi
OR PM020673 = 1	-- Polk - Owner - Mercedes
OR PM020651 = 1	-- Polk - Owner - BMW
OR PM021424 = 1	-- Polk - Owner - Lamborghini
OR PM020669 = 1	-- Polk - Owner - Land Rover
OR PM020663 = 1	-- Polk - Owner - Hyundai
OR PM020676 = 1	-- Polk - Owner - Nissan
OR PM020655 = 1	-- Polk - Owner - Chrysler Division
OR PM020656 = 1	-- Polk - Owner - Dodge Division
OR PM020667 = 1	-- Polk - Owner - Jeep
OR PM020681 = 1	-- Polk - Owner - Ram Truck
OR PM020688 = 1	-- Polk - Owner - Volkswagen
OR PM020668 = 1	-- Polk - Owner - KIA
OR PM020674 = 1	-- Polk - Owner - Mercury
OR PM021276 = 1	-- Polk - Owner - Rivian EV
OR PM020658 = 1	-- Polk - Owner - Ford Division
OR PM020671 = 1	-- Polk - Owner - Lincoln
)
AND snapshot = (
SELECT MAX(snapshot) FROM
dataproducts_dev.bronze_acxiom.gm_consumer_list
)
)


, GM_curr_owner_only AS (
SELECT *
FROM GM_curr_owner
WHERE gm_person_real_id
NOT IN 
(
SELECT gm_person_real_id
FROM non_GM_curr_owner
WHERE gm_person_real_id IS NOT NULL  
)
)

-- 16670213

